In [1]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
from pandas.plotting import scatter_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer



plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (15, 10)
plt.rcParams.update({
    "lines.color": "black",
    "patch.edgecolor": "black",
    "text.color": "black",
    "axes.facecolor": "white",
    "axes.edgecolor": "black",
    "axes.labelcolor": "black",
    "xtick.color": "black",
    "ytick.color": "black",
    "grid.color": "white",
    "figure.facecolor": "white",
    "figure.edgecolor": "white",
    "savefig.facecolor": "white",
    "savefig.edgecolor": "white",
    "font.size": 25})

%matplotlib inline

In [2]:
dataDir = "../data generation/Tstat/stdin/"
datafile = "/log_tcp_complete"
poolIPsFile = "../poolIP.txt"

In [3]:
## Aggregate Data
raw_df = pd.DataFrame()
for datafolder in sorted(os.listdir(dataDir)):
    dataPointFile = dataDir + datafolder + datafile
    raw_df = pd.concat([raw_df, pd.read_csv(dataPointFile, sep = ' ')])

In [4]:
raw_df = raw_df.reset_index(drop=True)

In [5]:
raw_df

,#09#c_ip:1,c_port:2,c_pkts_all:3,c_rst_cnt:4,c_ack_cnt:5,c_ack_cnt_p:6,c_bytes_uniq:7,c_pkts_data:8,c_bytes_all:9,c_pkts_retx:10,...,s_cwin_ini:95,s_pkts_rto:96,s_pkts_fs:97,s_pkts_reor:98,s_pkts_dup:99,s_pkts_unk:100,s_pkts_fc:101,s_pkts_unrto:102,s_pkts_unfs:103,s_syn_retx:104
0,192.168.0.92,43432,12,0,11,5,2473,5,2473,0,...,1460,0,0,1,0,0,0,0,0,0
1,192.168.0.92,43434,12,0,11,5,2595,5,2595,0,...,1460,0,0,1,0,0,0,0,0,0
2,192.168.0.92,45634,16,0,15,7,2999,7,5292,2,...,2920,0,0,1,0,2,0,0,0,0
3,192.168.0.92,45636,12,0,11,5,2765,5,2765,0,...,4435,0,0,1,0,1,0,0,0,0
4,192.168.0.92,47230,10,0,9,6,675,2,675,0,...,2920,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13150,192.168.0.92,57524,40,1,38,26,2645,11,2645,0,...,392,0,0,0,0,0,0,0,0,0
13151,192.168.0.92,57544,10,1,8,3,1354,4,1354,0,...,392,0,0,0,0,0,0,0,0,0
13152,192.168.0.92,57546,13,1,11,6,1354,4,1354,0,...,1448,0,0,1,0,0,0,0,0,0
13153,192.168.0.92,57548,13,1,11,6,1354,4,1354,0,...,2896,0,0,1,0,0,0,0,0,0


In [6]:
raw_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13155 entries, 0 to 13154
Columns: 104 entries, #09#c_ip:1 to s_syn_retx:104
dtypes: float64(17), int64(85), object(2)
memory usage: 10.4+ MB


In [7]:
f = open(dataDir + poolIPsFile, 'r')
poolIPs = f.read().splitlines()

In [8]:
def is_pool(x):
    return 1 if x in poolIPs else 0

raw_df["s_ip:15"] = raw_df["s_ip:15"].apply(is_pool)

In [9]:
raw_df.columns

Index(['#09#c_ip:1', 'c_port:2', 'c_pkts_all:3', 'c_rst_cnt:4', 'c_ack_cnt:5',
       'c_ack_cnt_p:6', 'c_bytes_uniq:7', 'c_pkts_data:8', 'c_bytes_all:9',
       'c_pkts_retx:10',
       ...
       's_cwin_ini:95', 's_pkts_rto:96', 's_pkts_fs:97', 's_pkts_reor:98',
       's_pkts_dup:99', 's_pkts_unk:100', 's_pkts_fc:101', 's_pkts_unrto:102',
       's_pkts_unfs:103', 's_syn_retx:104'],
      dtype='object', length=104)

In [10]:
bad_cols = ["#09#c_ip:1", "c_port:2", "s_port:16", 'c_pkts_ooo:12',
 'c_isint:38', 's_isint:39', 'c_iscrypto:40', 's_iscrypto:41', 'p2p_t:43',
  'c_ttl_min:50', 'c_ttl_max:51', 'c_f1323_opt:59', 'c_tm_opt:60',
   'c_sack_opt:62', 'c_mss:64', 'c_win_0:69', 'c_pkts_reor:75',
    'c_pkts_dup:76', 'c_pkts_fc:78', 'c_pkts_unrto:79',
     'c_pkts_unfs:80', 'c_syn_retx:81', 's_pkts_fc:101',
      's_pkts_unfs:103', 's_syn_retx:104']
important_cols = ['c_mss_min:66', 'c_cwin_min:71', 'durat:31', 's_last:35',
       'c_cwin_ini:72', 'c_ack_cnt_p:6', 's_pkts_all:17', 'c_last:34',
       's_ack_cnt_p:20', 's_ack_cnt:19', 'c_cwin_max:70', 's_cwin_ini:95',
       'c_bytes_uniq:7', 's_rtt_max:54', 's_cwin_max:93', 'c_pkts_data:8',
       'c_win_min:68', 's_cwin_min:94', 's_ttl_min:57', 's_bytes_retx:25']
pipeline_simple_features = ColumnTransformer([('drop_bad_cols','drop', bad_cols),
                                                ('scaler', StandardScaler(), important_cols),])

In [11]:
train_set, test_set = train_test_split(raw_df, test_size=0.2, random_state=42)

In [12]:
flow_df = pipeline_simple_features.fit_transform(train_set)
labels = train_set["s_ip:15"].copy()

In [13]:
test_flow_df = pipeline_simple_features.transform(test_set)
test_labels = test_set["s_ip:15"].copy()

# Decision Tree Classifier:

In [14]:
treeClassifier = DecisionTreeClassifier()
flow_pred_labels = cross_val_predict(treeClassifier, flow_df, labels, cv=10)
confusion_matrix(labels, flow_pred_labels)

array([[10327,     7],
       [    3,   187]])

In [15]:
print("Precision:", precision_score(labels, flow_pred_labels))
print("Recall:", recall_score(labels, flow_pred_labels))
print("F1:", f1_score(labels, flow_pred_labels))

Precision: 0.9639175257731959
Recall: 0.9842105263157894
F1: 0.9739583333333334


### Test Data on the Decision Tree Calssifier:

In [16]:
treeClassifier.fit(flow_df, labels)
test_pred_labels = treeClassifier.predict(test_flow_df)

In [17]:
print("Test Precision:", precision_score(test_labels, test_pred_labels))
print("Test Recall:", recall_score(test_labels, test_pred_labels))
print("Test F1:", f1_score(test_labels, test_pred_labels))

Test Precision: 1.0
Test Recall: 1.0
Test F1: 1.0


In [18]:
confusion_matrix(test_labels, test_pred_labels)

array([[2595,    0],
       [   0,   36]])

# Random Forrest Classifier:

In [19]:
randomforestClassifier = RandomForestClassifier(n_estimators=25)
flow_pred_labels = cross_val_predict(randomforestClassifier, flow_df, labels, cv=10)
confusion_matrix(labels, flow_pred_labels)

array([[10333,     1],
       [    1,   189]])

In [20]:
print("Precision:", precision_score(labels, flow_pred_labels))
print("Recall:", recall_score(labels, flow_pred_labels))
print("F1:", f1_score(labels, flow_pred_labels))

Precision: 0.9947368421052631
Recall: 0.9947368421052631
F1: 0.9947368421052631


### Test Data on Random Forrest Classifier:

In [21]:
randomforestClassifier.fit(flow_df, labels)
test_pred_labels = randomforestClassifier.predict(test_flow_df)

In [22]:
print("Test Precision:", precision_score(test_labels, test_pred_labels))
print("Test Recall:", recall_score(test_labels, test_pred_labels))
print("Test F1:", f1_score(test_labels, test_pred_labels))

Test Precision: 1.0
Test Recall: 1.0
Test F1: 1.0


In [23]:
confusion_matrix(test_labels, test_pred_labels)

array([[2595,    0],
       [   0,   36]])